In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import requests
import MetaTrader5 as mt

In [27]:
%%writefile .env

#
# (c) Ricky Macharm, MScFE
# https://SisengAI.com
#

LOGIN = 510005038
PASSWORD = "9t*nW*3Z" 
SERVER = "HantecMarkets-MT5"

DB_NAME = "cdf.db"

MODEL_DIRECTORY = "models"

Overwriting .env


In [22]:
%%writefile config.py

#
# (c) Ricky Macharm, MScFE
# https://SisengAI.com
#


"""This module extracts information from your `.env` file.
"""

# The os library allows you to communicate with a computer's
# operating system: https://docs.python.org/3/library/os.html
import os

# pydantic used for data validation: https://pydantic-docs.helpmanual.io/
from pydantic import BaseSettings


def return_full_path(filename: str = ".env") -> str:
    """Uses os to return the correct path of the `.env` file."""
    absolute_path = os.path.abspath(__file__)
    directory_name = os.path.dirname(absolute_path)
    full_path = os.path.join(directory_name, filename)
    return full_path


class Settings(BaseSettings):
    """Uses pydantic to define settings for project."""

    login: int
    password: str
    server: str

    db_name: str
    model_directory: str

    class Config:
        env_file = return_full_path(".env")


# Create instance of `Settings` class that will be imported
# in lesson notebooks and the other modules for application.
settings = Settings()


Overwriting config.py


In [1]:
# import settings
from config import settings

# use 'dir' to check out the attributes
print(settings.server)

HantecMarkets-MT5


In [22]:
%%writefile hantec_MT5.py

#
# (c) Ricky Macharm, MScFE
# https://SisengAI.com
#



import numpy as np
import pandas as pd
from config import settings
import MetaTrader5 as mt



mt.initialize()
 
# log in to trading account
mt.login(settings.login, settings.password, settings.server)

def hantec_data(ticker, tf="D1", start_bar = 0, num_bars = 99999):
    
    """Code to get the desired trading instrument, time frame and the 
    features of 'open', 'high', 'low' and 'close ' arranged in a convenient fashion
    from the mt5 platform and returned as a DataFrame"""
    
    ticker_list = [s.name for s in mt.symbols_get()]
    if ticker not in ticker_list:
                raise Exception(
                    f"Invalid Ticker name. {settings.server} does not contain '{ticker}'."
                   # f"Here is a list of all the tickers found: {print(pd.Series(ticker_list).to_string())}"
                )
    
    timeframe = eval(f'mt.TIMEFRAME_{tf}') # to remove the quotation marks
    bars = mt.copy_rates_from_pos(ticker, timeframe, start_bar, num_bars)
        
    return (pd.DataFrame(bars)
         .assign(date = lambda x: 
                 pd.to_datetime(x['time'], unit='s'))
         .drop(['spread', 'real_volume', 'time'], axis=1)
         #.pipe(lambda x: x.loc[x.tick_volume > 10])
         .set_index('date')
        )  
    
tickers = ['AUDUSD', 'EURJPY', 'EURUSD', 'GBPJPY', 'GBPUSD', 'USDCAD', 'USDCHF', 'USDJPY', 
        'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'CADCHF', 'CADJPY', 'CHFJPY', 'EURAUD',
        'EURCAD', 'EURCHF', 'EURGBP', 'EURNZD', 'GBPAUD', 'GBPCAD', 'GBPCHF', 'GBPNZD', 'US500',
        'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD', 'XAUUSD', 'US30',  'US100', 'GER40', 'UK100',]
    


Overwriting hantec_MT5.py


In [1]:
from hantec_MT5 import *

ModuleNotFoundError: No module named 'MetaTrader5'

In [8]:
hantec_data("EURUSD")

,open,high,low,close,tick_volume
date,,,,,
1971-01-04,0.53690,0.53690,0.53690,0.53690,1
1971-01-05,0.53660,0.53660,0.53660,0.53660,1
1971-01-06,0.53650,0.53650,0.53650,0.53650,1
1971-01-07,0.53680,0.53680,0.53680,0.53680,1
1971-01-08,0.53710,0.53710,0.53710,0.53710,1
...,...,...,...,...,...
2023-02-28,1.06083,1.06451,1.05741,1.05750,61847
2023-03-01,1.05755,1.06912,1.05653,1.06666,77008
2023-03-02,1.06660,1.06728,1.05765,1.05968,66760


In [9]:
mt.symbols_total()

2488

In [10]:
# display data on the MetaTrader 5 package
print(f"MetaTrader5 package author: {mt.__author__}")
print(f"MetaTrader5 package version: {mt.__version__}")

MetaTrader5 package author: MetaQuotes Ltd.
MetaTrader5 package version: 5.0.37


In [11]:
tickers=mt.symbols_get()
print('Tickers: ', len(tickers))

Tickers:  2488


In [6]:
count=0
# display the first five ones
for s in tickers:
    count+=1
    print(f"{count}. {s.name}")
    if count==5: break
print()

1. EURUSD
2. AUDUSD
3. EURGBP
4. GBPUSD
5. NZDUSD



In [12]:
type(tickers)

tuple

In [ ]:
ticker_list = [s.name for s in tickers]
ticker_list

In [ ]:
ticker_list = []
for s in tickers:
    ticker_list.append(s.name)
    
ticker_list